In [78]:
import pandas as pd
import pickle
import MySQLdb

In [79]:
def get_data(date):
    
    article_df, comment_df = "", ""
    
    with open("./data/article_" + date + ".plk", 'rb') as file:
        article_df = pickle.load(file)
        
    with open("./data/comment_" + date + ".plk", 'rb') as file:
        comment_df = pickle.load(file)
        
    return article_df, comment_df

In [117]:
date = "2016-06-01"
article_df, comment_df = get_data(date)
len(article_df), len(comment_df)

(2327, 117663)

In [109]:
db = MySQLdb.connect(
    "127.0.0.1",
    "root",
    "****",
    "naver_news",
    charset='utf8',
)

In [110]:
def make_article_insert(data):
    QUERY = """
        INSERT INTO article (newsid, oid, newspaper, title, link, comment, likeit, content, date, category)                   
        VALUES ("{newsid}", "{oid}", "{newspaper}", "{title}", "{link}", {comment}, {likeit}, "{content}", "{date}", {category})
    """.format(
        newsid=data["newsid"], oid=data["oid"], newspaper=data["newspaper"], title=data["title"].replace("\"","'"), link=data["link"], comment=data["comment"], likeit=data["likeit"], content=data["content"].replace("\"","'"), date=data["date"], category=data["category"])
    return QUERY.replace("\n","")

In [111]:
def insert_article(article_df):
    for idx in range(len(article_df)):

        if idx%1000 == 0:
            print(idx)

        query = make_article_insert(article_df.loc[idx])
        cursor = db.cursor()
        cursor.execute(query)
        db.commit()

In [112]:
%time insert_article(article_df)

0
1000
2000
CPU times: user 1.24 s, sys: 136 ms, total: 1.37 s
Wall time: 2.7 s


In [113]:
def replace_str_list(emoticon_list, content):
    emoticon_list = list(emoticon_list)
    for emoticon in emoticon_list:
        content = content.replace(emoticon, "")
    return content

def make_comment_insert(data):
    
    emoticon_list = "🙋🐷😱👏😨🏈😈👿😇😆😉😀😁😃😄😲😞😗😝😗😒😲😛😕😠😬😡😎😚😘😙😟😲😤😆😊😢😯😇😂😅😧🐖🐒💨💚🐶🐕🐽👎🎊✌😉👌👍😜"
    emoticon_list += "👬👭🍷🍹☕🍨🍴🍰🍗🍞🍜🍚🍛🍲🍎🍏🍊🍋🍄🍅🍇🍓🍍🍒🍑🍐🍈👒👗👜👚👡👠👙👝👛💰💳🐔🐥🔫✈🔪🍚🍖🍗👽👾💀👹👺😂🖕🏿🎴💕🐦"
    emoticon_list += "👥🍄🍆🍑🌹🙏🐀🐁💂🐔✈➡👳😧😵😆❌👅💩👊😠😦🐥🐣🙉😊😀😁😃😄😆😉😛😜😝😋☺🐗🍖🍴👍🚗🚕🚘👌🏻🔥❎🙈🙉🙊🤔😓😍💥🎶🚔"
    emoticon_list += "🇰🇮🇱🇮🇱🇧🇪🏥😔🐩💉🍆🍉🌰💖🎵💯😭🌷😷💦😥😑😐😰😧😤😣👲🐢🐰🙇😩👉💣👈"
        
    contents = data["contents"].replace("\"","'").replace("\\","")
    contents = replace_str_list(emoticon_list, contents)
    
    QUERY = """
        INSERT INTO comments (category, oid, userIdNo, userName, good, bad, contents, regTime)                   
        VALUES ({category}, "{oid}", "{userIdNo}", "{userName}", {good}, {bad}, "{contents}", "{regTime}")
    """.format(
        category=data["category"], aid=data["aid"], oid=data["oid"], userIdNo=data["userIdNo"], userName=data["userName"], good=data["good"], bad=data["bad"], contents=contents, regTime=data["regTime"])
    return QUERY.replace("\n","")

In [115]:
def insert_comment(comment_df, start=0):
    
    for idx in range(start, len(comment_df)):

        if idx%1000 == 0:
            print(idx)

        query = make_comment_insert(comment_df.loc[idx])

#         print(idx, comment_df.loc[idx]["contents"], end="\n\n")

        cursor = db.cursor()
        cursor.execute(query)
            
        db.commit()

In [118]:
# %time insert_comment(comment_df)

In [119]:
db.commit()